In [1]:
import numpy as np
import pandas as pd

## Data import:

In [2]:
ctq_maws_a1_df = pd.read_csv("CTQ_20191008.csv")
ctq_maws_a1_df = ctq_maws_a1_df[['redcap_id'] + [col for col in ctq_maws_a1_df if col.startswith('ctq')]]

In [3]:
ctq_maws_a1_df.head(2)

,redcap_id,ctq_your_childhood_timestamp,ctqq_1,ctqq_2,ctqq_3,ctqq_4,ctqq_5,ctqq_6,ctqq_7,ctqq_8,...,ctqq_20,ctqq_21,ctqq_22,ctqq_23,ctqq_24,ctqq_25,ctqq_26,ctqq_27,ctqq_28,ctq_your_childhood_complete
0,E1,NaN,1.0,5.0,3.0,1.0,5.0,1.0,5.0,1.0,...,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,5.0,2
1,E2,13-08-19 10:36,4.0,3.0,5.0,3.0,1.0,1.0,2.0,5.0,...,1.0,1.0,2.0,2.0,1.0,4.0,5.0,1.0,1.0,2


In [4]:
ctq_maws_a1_df.shape

(84, 31)

### Set index to redcap_id:

In [5]:
ctq_maws_a1_df = ctq_maws_a1_df.set_index('redcap_id')

## Check and remove duplicated redcap_ids:

In [6]:
ctq_maws_a1_df[ctq_maws_a1_df.index.duplicated()]

,ctq_your_childhood_timestamp,ctqq_1,ctqq_2,ctqq_3,ctqq_4,ctqq_5,ctqq_6,ctqq_7,ctqq_8,ctqq_9,...,ctqq_20,ctqq_21,ctqq_22,ctqq_23,ctqq_24,ctqq_25,ctqq_26,ctqq_27,ctqq_28,ctq_your_childhood_complete
redcap_id,,,,,,,,,,,,,,,,,,,,,


In [7]:
ctq_maws_a1_df = ctq_maws_a1_df[~ctq_maws_a1_df.index.duplicated(keep ='first')]

## Subsetting data to questionnaire answers only:

In [8]:
ctq_maws_a1_subset_df = ctq_maws_a1_df.iloc[:, 1:len(ctq_maws_a1_df.columns)-1]

In [9]:
ctq_maws_a1_subset_df.head(2)

,ctqq_1,ctqq_2,ctqq_3,ctqq_4,ctqq_5,ctqq_6,ctqq_7,ctqq_8,ctqq_9,ctqq_10,...,ctqq_19,ctqq_20,ctqq_21,ctqq_22,ctqq_23,ctqq_24,ctqq_25,ctqq_26,ctqq_27,ctqq_28
redcap_id,,,,,,,,,,,,,,,,,,,,,
E1,1.0,5.0,3.0,1.0,5.0,1.0,5.0,1.0,1.0,5.0,...,5.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,5.0
E2,4.0,3.0,5.0,3.0,1.0,1.0,2.0,5.0,5.0,1.0,...,1.0,1.0,1.0,2.0,2.0,1.0,4.0,5.0,1.0,1.0


In [10]:
ctq_maws_a1_subset_df.shape

(84, 28)

## Replace 99's (no answer) with NaNs:

In [11]:
ctq_maws_a1_subset_df = ctq_maws_a1_subset_df.replace(99, np.NaN)

## Check if any patient has not filled the questionnaire and remove:

In [12]:
ctq_maws_empty_l = ctq_maws_a1_subset_df[ctq_maws_a1_subset_df.isnull().all(1)].index.tolist()
ctq_maws_empty_l

['E7', 'E29', 'E35', 'F12', 'F13', 'F18', 'F27']

In [13]:
ctq_maws_a1_subset_df = ctq_maws_a1_subset_df.drop(ctq_maws_empty_l)

## Check if all values are between 1 and 5 and return redcap_ids values that are not:

In [14]:
[x for x in np.unique(ctq_maws_a1_subset_df) if not np.isnan(x)]

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [15]:
# out_of_bounds_bool_s = ctw_maws_a1_subset_df[~((ctw_maws_a1_subset_df >= 1) & (ctw_maws_a1_subset_df <= 5))].T.any()
# out_of_bounds_bool_s[out_of_bounds_bool_s]

## Get Total Subscales Scores per Patient DataFrame and save to file:

In [16]:
# Name subscales and assign respective question ids:
subscales_d = {"emotional_abuse": [3,8,14,18,25], "physical_abuse": [9,11,12,15,17],
            "sexual_abuse": [20,21,23,24,27], "emotional_neglect": [5,7,13,19,28],
            "physical_neglect": [1,2,4,6,26], "minimization_denial": [10,16,22]}

# Add ctqq text:
for key, value in subscales_d.items():
    subscales_d[key] = ["ctqq_" + str(i) for i in value]

### Get and reverse reverse coded questions:

In [17]:
def reverse_score(x):
    """
    Funciton to reverse-code scores using indexes from list below:
    """
    score_values_l = [1,2,3,4,5]
    
    if not np.isnan(x):
    
        try:
            return score_values_l[-int(x)]
        except IndexError:
            print(x)

In [18]:
reverse_coded_questions_l = [5,7,13,19,28,2,26]
reverse_l = ["ctqq_" + str(i) for i in reverse_coded_questions_l]

In [19]:
ctq_maws_a1_subset_df[reverse_l] = ctq_maws_a1_subset_df[reverse_l].copy().applymap(reverse_score)

6.0


In [20]:
# Total scores are computed in new DataFrame here by taking:
# ( sum of scores in subscale / ( amount of questions in subscale - unanswered questions(NaN) ) )

total_scores_df = pd.DataFrame(pd.DataFrame(index=ctq_maws_a1_subset_df.index))

for key in subscales_d.keys():
    total_scores_df[key+"_prorated"] = ctq_maws_a1_subset_df[subscales_d[key]].sum(axis=1)/(len(ctq_maws_a1_subset_df[subscales_d[key]].columns)-ctq_maws_a1_subset_df[subscales_d[key]].isnull().sum(axis=1))
    total_scores_df[key+"_sum"] = ctq_maws_a1_subset_df[subscales_d[key]].sum(axis=1)
    total_scores_df[key+"_contains_nan"] = ctq_maws_a1_subset_df[subscales_d[key]].isnull().any(axis=1)
    total_scores_df[key+"_nan_counts"] = ctq_maws_a1_subset_df[subscales_d[key]].isnull().sum(axis=1)
    
total_scores_df["contains_nan"] = ctq_maws_a1_subset_df.isnull().any(axis=1)

In [22]:
# Add rank columns:
def assign_rank(row):
    score_sum = row.emotional_abuse_sum
    if score_sum >= 5 and score_sum <= 8:
        emotional_abuse_rank = "None"
    elif score_sum >= 9 and score_sum <= 12:
        emotional_abuse_rank = "Low"
    elif score_sum >= 13 and score_sum <= 15:
        emotional_abuse_rank = "Moderate"
    elif score_sum >= 16:
        emotional_abuse_rank = "High"
    else:
        emotional_abuse_rank = np.nan
            
    score_sum = row.physical_abuse_sum
    if score_sum >= 5 and score_sum <= 7:
        physical_abuse_rank = "None"
    elif score_sum >= 8 and score_sum <= 9:
        physical_abuse_rank = "Low"
    elif score_sum >= 10 and score_sum <= 12:
        physical_abuse_rank = "Moderate"
    elif score_sum >= 13:
        physical_abuse_rank = "High"
    else:
        physical_abuse_rank = np.nan
            
    score_sum = row.sexual_abuse_sum
    if score_sum == 5:
        sexual_abuse_rank = "None"
    elif score_sum >= 6 and score_sum <= 7:
        sexual_abuse_rank = "Low"
    elif score_sum >= 8 and score_sum <= 12:
        sexual_abuse_rank = "Moderate"
    elif score_sum >= 13:
        sexual_abuse_rank = "High"
    else:
        sexual_abuse_rank = np.nan
            
    score_sum = row.emotional_neglect_sum
    if score_sum >= 5 and score_sum <= 9:
        emotional_neglect_rank = "None"
    elif score_sum >= 10 and score_sum <= 14:
        emotional_neglect_rank = "Low"
    elif score_sum >= 15 and score_sum <= 17:
        emotional_neglect_rank = "Moderate"
    elif score_sum >= 18:
        emotional_neglect_rank = "High"
    else:
        emotional_neglect_rank = np.nan
            
    score_sum = row.physical_neglect_sum
    if score_sum >= 5 and score_sum <= 7:
        physical_neglect_rank = "None"
    elif score_sum >= 8 and score_sum <= 9:
        physical_neglect_rank = "Low"
    elif score_sum >= 10 and score_sum <= 12:
        physical_neglect_rank = "Moderate"
    elif score_sum >= 13:
        physical_neglect_rank = "High"
    else:
        physical_neglect_rank = np.nan
        
    return pd.Series((emotional_abuse_rank, physical_abuse_rank, sexual_abuse_rank, 
                      emotional_neglect_rank, physical_neglect_rank))

In [23]:
total_scores_df[['emotional_abuse_rank', 'physical_abuse_rank', 'sexual_abuse_rank', 
                 'emotional_neglect_rank', 'physical_neglect_rank']] = total_scores_df.apply(assign_rank, axis=1)

In [24]:
total_scores_df = total_scores_df.reindex(sorted(total_scores_df.columns), axis=1)

In [25]:
total_scores_df.round(1)

,contains_nan,emotional_abuse_contains_nan,emotional_abuse_nan_counts,emotional_abuse_prorated,emotional_abuse_rank,emotional_abuse_sum,emotional_neglect_contains_nan,emotional_neglect_nan_counts,emotional_neglect_prorated,emotional_neglect_rank,...,physical_neglect_contains_nan,physical_neglect_nan_counts,physical_neglect_prorated,physical_neglect_rank,physical_neglect_sum,sexual_abuse_contains_nan,sexual_abuse_nan_counts,sexual_abuse_prorated,sexual_abuse_rank,sexual_abuse_sum
redcap_id,,,,,,,,,,,,,,,,,,,,,
E1,False,False,0,2.2,Low,11.0,False,0,1.0,None,...,False,0,1.0,None,5.0,False,0,1.0,None,5.0
E2,False,False,0,4.8,High,24.0,False,0,4.8,High,...,False,0,2.4,Moderate,12.0,False,0,1.2,Low,6.0
E3,False,False,0,1.0,None,5.0,False,0,4.2,High,...,False,0,3.0,High,15.0,False,0,1.2,Low,6.0
E4,False,False,0,1.0,None,5.0,False,0,2.8,Low,...,False,0,1.2,None,6.0,False,0,1.2,Low,6.0
E5,False,False,0,1.0,None,5.0,False,0,1.0,None,...,False,0,1.0,None,5.0,False,0,1.0,None,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F29,False,False,0,1.6,None,8.0,False,0,2.2,Low,...,False,0,1.2,None,6.0,False,0,1.0,None,5.0
F30,False,False,0,1.6,None,8.0,False,0,3.0,Moderate,...,False,0,1.2,None,6.0,False,0,1.0,None,5.0
F31,False,False,0,3.4,High,17.0,False,0,2.4,Low,...,False,0,1.0,None,5.0,False,0,2.0,Moderate,10.0


### Save dataframe to csv file:

In [26]:
total_scores_df.round(1).to_csv("CTQ_MAWS_total_subscale_scores_ALL.csv")